In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
import sympy as sp

In [2]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [4]:
df = pd.read_csv('hundo_equations.csv')
functions = []
for _, row in df.iterrows():
    formula = row['Formula']
    num_vars = row['# parameters']
    
    function_details = {
        'formula': formula,
        'num_variables': num_vars,
        'parameters': []
    }
    
    for i in range(1, num_vars + 1):
        param_name = row.get(f'p{i}_name', None)
        if pd.notna(param_name):
            function_details['parameters'].append(param_name)
    
    functions.append(function_details)

In [5]:
high = 1
low = -1
sequence_size = 1000
num_funcs = 10
max_vars = max([f['num_variables'] for f in functions])
sample_size = num_funcs*100

x = torch.linspace(high, low, sequence_size)
# x = x.unsqueeze(1).expand(sequence_size, int(sample_size/num_funcs))

param_values = []
for func in functions:
    hold = []
    for param in func['parameters']:
        p = (high - low) * torch.rand(sample_size) + low
        hold.append(p)
    params = torch.stack(hold)
    params = F.pad(params, (0, 0,0,max_vars - params.shape[0]))
    param_values.append(params)
param_values = torch.stack(param_values)

print(x.shape)
print(param_values.shape)

torch.Size([1000])
torch.Size([100, 3, 1000])


In [7]:
y_values = []
symbols = []
for f in range(len(functions)):
    formula = sp.sympify(functions[f]['formula'])
    syms = [sp.symbols(p) for p in functions[f]['parameters']]
    syms.append(sp.symbols('x'))
    symbols.append(syms)
    y = []
    for i in range(sample_size):
        var_values = {syms[n]: float(param_values[f, n, i]) for n in range(len(syms)-1)}
        var_values[syms[-1]] = float(x[i])
        evaluate = formula.subs(var_values)
        print(torch.tensor(float(evaluate)))
        y.append(torch.tensor(float(evaluate)))
    y_values.append(torch.stack(y))
y_values = torch.stack(y_values)
y_values.shape

tensor(-2.0494)
tensor(0.9288)
tensor(0.1504)
tensor(-0.2652)
tensor(-1.1453)
tensor(0.8024)
tensor(-2.4228)
tensor(0.2912)
tensor(-0.9908)
tensor(-0.3373)
tensor(0.3464)
tensor(-1.5048)
tensor(-0.0311)
tensor(-0.2048)
tensor(-1.0048)
tensor(0.6221)
tensor(-0.8638)
tensor(0.5826)
tensor(-0.6079)
tensor(1.7645)
tensor(-0.2347)
tensor(-0.1138)
tensor(0.3577)
tensor(-0.2912)
tensor(-1.5540)
tensor(0.5548)
tensor(0.3323)
tensor(-1.8883)
tensor(-0.9161)
tensor(-0.2013)
tensor(-0.0547)
tensor(-1.3273)
tensor(0.6175)
tensor(-0.3190)
tensor(0.4634)
tensor(-0.2773)
tensor(0.5286)
tensor(-1.3319)
tensor(0.1374)
tensor(-0.7491)
tensor(-1.8025)
tensor(0.0387)
tensor(-0.6096)
tensor(0.6638)
tensor(0.8501)
tensor(-0.0561)
tensor(1.1055)
tensor(1.5025)
tensor(-0.8718)
tensor(0.2916)
tensor(0.1145)
tensor(0.8319)
tensor(0.6737)
tensor(-0.3683)
tensor(2.1045)
tensor(0.5138)
tensor(-0.4096)
tensor(1.9750)
tensor(0.1893)
tensor(-1.0032)
tensor(-0.5970)
tensor(-1.2928)
tensor(-1.4780)
tensor(1.1814)
tenso

TypeError: Cannot convert expression to float